<a href="https://colab.research.google.com/github/carlosks/carlosks/blob/main/Teste_de_finetuning_summarizer_Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-wlx1aoq3/unsloth_8846083f97ef41d5b0a89c9d8b078cd2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-wlx1aoq3/unsloth_8846083f97ef41d5b0a89c9d8b078cd2
  Resolved https://github.com/unslothai/unsloth.git to commit 0fbbdfc091fc1a3b1c09b752794963681d10fad2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
!pip install --upgrade accelerate
!pip install --upgrade transformers
!pip install --upgrade peft
!pip install --upgrade bitsandbytes
!pip install --upgrade trl
!pip install --upgrade datasets


In [2]:
!pip install triton

In [3]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/drive/MyDrive/Desafio_Fase_3/arquivos 40 Mega/titles_dataset_chat_data.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Desafio_Fase_3/arquivos 40 Mega/formated_titles_dataset_chat_data.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [4]:
import json



def format_dataset_into_model_input(data):

    def separate_text(full_text):

        # Encontrar os delimitadores

        news_start = full_text.find("[|title|]") + len("[|title|]")

        news_end = full_text.find("[|etitle|]")

        summary_start = full_text.find("[|content|]") + len("[|content|]")

        summary_end = full_text.find("[|econtent|]")



        # Verificar se todos os delimitadores foram encontrados

        if news_start == -1 or news_end == -1 or summary_start == -1 or summary_end == -1:

            print(f"Delimitador faltando no texto: {full_text[:50]}...")

            return None, None, None



        # Extrair a instrução, texto de entrada e resposta

        instruction = full_text.split('\n')[0]  # Primeira linha é a instrução

        input_text = full_text[news_start:news_end].strip()  # Extrair texto entre [|News|] e [|eNews|]

        response = full_text[summary_start:summary_end].strip()  # Extrair resumo entre [|summary|] e [|esummary|]



        return instruction, input_text, response



    # Inicializando as listas para armazenar os dados

    instructions = []

    inputs = []

    outputs = []



    # Processando o dataset

    try:

        for item in data:

            # Acessar diretamente o campo 'input' e extrair o conteúdo

            instruction, input_text, response = separate_text(item['input'])

            if instruction and input_text and response:  # Verificar se todos os textos são válidos

                instructions.append(instruction)

                inputs.append(input_text)

                outputs.append(response)

    except KeyboardInterrupt:

        pass  # Ignorar interrupções de teclado



    # Criar o dicionário final

    formatted_data = {

        "instruction": instructions,

        "input": inputs,

        "output": outputs

    }



    # Salvar o resultado em um arquivo JSON

    with open(OUTPUT_PATH_DATASET, 'w') as output_file:

        json.dump(formatted_data, output_file, indent=4)

In [5]:
# Load the dataset
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

format_dataset_into_model_input(data)


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Desafio_Fase_3/arquivos 40 Mega/formated_titles_dataset_chat_data.json"

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/35998 [00:00<?, ? examples/s]

In [9]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/35998 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 35,998 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.418000
2,3.562500
3,3.559100
4,3.715300
5,3.448300


OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 101.06 MiB is free. Process 206428 has 14.65 GiB memory in use. Of the allocated memory 13.74 GiB is allocated by PyTorch, and 766.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [8]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS CONTENT",
        "Like the similar books about Australia, Hawaii, and Japan, each of these offers multiple photographic impressions recorded on a single day. Canada, for coverage of so vast a territory, engaged 100 international photographers (to Israelis ' mostly native 55), to produce 100,000 images (to Israelis ' 40,000), from which its several hundred memorable black-and-white and color illustrations were chosen. The photos juxtapose the industrial to the rural. A lively essay recounts experiences organizing the project and photographing on the actual day (June 8, 1984). The photographers get credit next to their pictures; a map shows where each worked that day; and a lengthy section gives their respective backgrounds, illustrated with past work. Israelis seems earnest, even somber, in comparison. With more than 300 equally fine images, it plays down the individual artists, presenting only a list of credits at the end. The introduction discusses not so much this project as the past and present social and political developments in this young nation, placing the images firmly in context. And indeed, throughout the coverage of agriculture, sheep, beaches, and apartment houses are the ubiquitous soldiers with guns. The heat and aridity are everywhere evident on this day (May 10, 1984) in the ``green'' month, before things are parched to brown. Both books use a chronological arrangement and tell when, to the hour, the pictures were taken, but Canada , being so large, offers maps on each page to show the geographical location of particular images. Both recommended for their candid presentation of ordinary daily life. Kathleen Collins, Library of CongressCopyright 1985 Reed Business Information, Inc.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nSUMMARIZE THIS CONTENT\n\n### Input:\nLike the similar books about Australia, Hawaii, and Japan, each of these offers multiple photographic impressions recorded on a single day. Canada, for coverage of so vast a territory, engaged 100 international photographers (to Israelis'mostly native 55), to produce 100,000 images (to Israelis'40,000), from which its several hundred memorable black-and-white and color illustrations were chosen. The photos juxtapose the industrial to the rural. A lively essay recounts experiences organizing the project and photographing on the actual day (June 8, 1984). The photographers get credit next to their pictures; a map shows where each worked that day; and a lengthy section gives their respective backgrounds, illustrated with past work. Israelis seems earnest, e

In [12]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS TITLE",
        "Like the similar books about Australia, Hawaii, and Japan, each of these offers multiple photographic impressions recorded on a single day. Canada, for coverage of so vast a territory, engaged 100 international photographers (to Israelis ' mostly native 55), to produce 100,000 images (to Israelis ' 40,000), from which its several hundred memorable black-and-white and color illustrations were chosen. The photos juxtapose the industrial to the rural. A lively essay recounts experiences organizing the project and photographing on the actual day (June 8, 1984). The photographers get credit next to their pictures; a map shows where each worked that day; and a lengthy section gives their respective backgrounds, illustrated with past work. Israelis seems earnest, even somber, in comparison. With more than 300 equally fine images, it plays down the individual artists, presenting only a list of credits at the end. The introduction discusses not so much this project as the past and present social and political developments in this young nation, placing the images firmly in context. And indeed, throughout the coverage of agriculture, sheep, beaches, and apartment houses are the ubiquitous soldiers with guns. The heat and aridity are everywhere evident on this day (May 10, 1984) in the ``green'' month, before things are parched to brown. Both books use a chronological arrangement and tell when, to the hour, the pictures were taken, but Canada , being so large, offers maps on each page to show the geographical location of particular images. Both recommended for their candid presentation of ordinary daily life. Kathleen Collins, Library of CongressCopyright 1985 Reed Business Information, Inc.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS TITLE

### Input:
Like the similar books about Australia, Hawaii, and Japan, each of these offers multiple photographic impressions recorded on a single day. Canada, for coverage of so vast a territory, engaged 100 international photographers (to Israelis'mostly native 55), to produce 100,000 images (to Israelis'40,000), from which its several hundred memorable black-and-white and color illustrations were chosen. The photos juxtapose the industrial to the rural. A lively essay recounts experiences organizing the project and photographing on the actual day (June 8, 1984). The photographers get credit next to their pictures; a map shows where each worked that day; and a lengthy section gives their respective backgrounds, illustrated with past work. Israelis seems earnest, even somber

In [13]:
model.save_pretrained("/content/drive/MyDrive/Desafio_Fase_3/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Desafio_Fase_3/lora_model")


('/content/drive/MyDrive/Desafio_Fase_3/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Desafio_Fase_3/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Desafio_Fase_3/lora_model/tokenizer.json')

In [15]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/Desafio_Fase_3/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS TITLE",
        "Like the similar books about Australia, Hawaii, and Japan, each of these offers multiple photographic impressions recorded on a single day. Canada, for coverage of so vast a territory, engaged 100 international photographers (to Israelis ' mostly native 55), to produce 100,000 images (to Israelis ' 40,000), from which its several hundred memorable black-and-white and color illustrations were chosen. The photos juxtapose the industrial to the rural. A lively essay recounts experiences organizing the project and photographing on the actual day (June 8, 1984). The photographers get credit next to their pictures; a map shows where each worked that day; and a lengthy section gives their respective backgrounds, illustrated with past work. Israelis seems earnest, even somber, in comparison. With more than 300 equally fine images, it plays down the individual artists, presenting only a list of credits at the end. The introduction discusses not so much this project as the past and present social and political developments in this young nation, placing the images firmly in context. And indeed, throughout the coverage of agriculture, sheep, beaches, and apartment houses are the ubiquitous soldiers with guns. The heat and aridity are everywhere evident on this day (May 10, 1984) in the ``green'' month, before things are parched to brown. Both books use a chronological arrangement and tell when, to the hour, the pictures were taken, but Canada , being so large, offers maps on each page to show the geographical location of particular images. Both recommended for their candid presentation of ordinary daily life. Kathleen Collins, Library of CongressCopyright 1985 Reed Business Information, Inc.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)